In [ ]:
# Updated: ignore the beginnings of recordings prior to the beep

In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

import os
import glob
import time
import warnings
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import KFold


# Mask value (if all features for a given sample timestep are equal to MASK_VALUE, 
# then the sample timestep will be masked (skipped))
# Cannot use MASK_VALUE=0.0, as it corresponds to no movement (derivatives are zero)
# Cannot use MASK_VALUE=np.inf, as RandomUnderSampler cannot handle NaNs and Inf values
MASK_VALUE = 7777777.7777777
    
    
def segment_dataset(n_folds, selected_features, window_size, val_size):
    '''
    Segment all folds and all train/val/test partitions of the dataset.
    
    Dataset is segmented into same-length (window_size) sequences.
    Feature segments are pre-padded with MASK_VALUE-s and label segments with 0 (not a nod).
    One output file saved for each fold. 
    ''' 

    dataset_split_filename = f'/home/ICT2000/jondras/datasets/vra1/subject_independent/dataset_split_{n_folds}fold.npz'
    dataset_output_filename_prefix = f'/home/ICT2000/jondras/datasets/vra1/subject_independent/'
    nods_offsets_filename = f'/home/ICT2000/jondras/datasets/vra1/offsetListenerNods.txt'
    nods_offsets = np.loadtxt(nods_offsets_filename)
    
    # Cleaned version: ignore the beginnings of recordings prior to the beep
    dataset_type = f'C_{window_size}ws_{len(selected_features)}f'
    # Without ignoring the beginnings of recordings prior to the beep
#     dataset_type = f'{window_size}ws_{len(selected_features)}f'
    # For feature sets without translations
#     dataset_type = f'R_{window_size}ws_{len(selected_features)}f'
    dataset_output_filename_prefix += dataset_type + '/'
    if not os.path.exists(dataset_output_filename_prefix):
        os.makedirs(dataset_output_filename_prefix)
    
    n_features = len(selected_features)
    print(f'Selected features: \n\t{selected_features}')
    
    
    def get_croppped_dataframe(annotated_features_file):
        '''
        Ignore the beginning of the recording prior to the beep & take only selected features and annotations.
        '''
        
        # Get subject ID and its offset
        sid = annotated_features_file.split('/')[-1].split('.')[0][3:]
        offset = nods_offsets[int(sid) - 1]
#         print(sid, offset)
        
        df = pd.read_csv(annotated_features_file)
        df = df[df[' timestamp'] >= offset][selected_features + ['nod']]
        
        return df
    
    
    def get_segments(df):
        '''
        Generate segments (X (features) and Y (labels)) from the dataframe. 
        
        Returns 2 lists of 2D arrays.
        '''
        
        X = []
        Y = []

        # Pre-pad all features and labels with (window_size - 1) MASK_VALUE-s 
        padded_features = np.pad(df.values[:, :-1], ((window_size - 1, 0), (0, 0)), 
                                 mode='constant', constant_values=(MASK_VALUE, MASK_VALUE))
        # Labels are padded with 0 mask value (indicating not a nod)
        padded_labels   = np.pad(df.values[:, -1],  (window_size - 1, 0), 
                                 mode='constant', constant_values=(0, 0))
        
        assert padded_features.shape[1] == n_features
        assert padded_labels.shape[0] == padded_features.shape[0]
        assert len(padded_features) - window_size + 1 == len(df), 'Padding failed!'

        # Slide window of length window_size over the padded features/labels
        for i in range(len(df)):       
            X.append( padded_features[i:i + window_size] )
            Y.append( padded_labels[i:i + window_size] )

        return X, Y
    
    
    # Load the dataset split
    dataset_split = np.load(dataset_split_filename)
    start_time = time.time()
    for k in range(n_folds):
        
        segments = defaultdict(list)
        
        # Segment the train+val set
        for annotated_features_file in dataset_split[f'{k}_trainval']:
            df = get_croppped_dataframe(annotated_features_file)
                                   
            # Split recording into train and val partitions
            # Validation part starts at a random start index and has the length int(val_size * len(df))
            # start_idx (included), end_idx (excluded)
            val_len = int(val_size * len(df))
            start_idx = np.random.randint(0, len(df) - val_len + 1)
            if val_len < window_size:
                warnings.warn(f'Validation size {val_len} is less than {window_size}!')
            
            # Get segments from the val partition
            X_val, Y_val = get_segments(df=df.iloc[start_idx:start_idx + val_len])
            # Get segments from the LHS of the val partition
            X_train_1, Y_train_1 = get_segments(df=df.iloc[0:start_idx])
            # Get segments from the RHS of the val partition
            X_train_2, Y_train_2 = get_segments(df=df.iloc[start_idx + val_len:len(df)])

            assert len(X_val) == val_len
            assert len(X_train_1) + len(X_train_2) == len(df) - val_len
            
            segments['X_train'].extend(X_train_1)
            segments['X_train'].extend(X_train_2)
            segments['X_val'].extend(X_val)
            
            segments['Y_train'].extend(Y_train_1)
            segments['Y_train'].extend(Y_train_2)
            segments['Y_val'].extend(Y_val)
            
            # Record lengths of each batch of segments (needed for correct evaluation)
            segments['train_len'].append((len(X_train_1), len(X_train_2)))
            segments['val_len'].append(len(X_val))
        
        # Segment the test set
        for annotated_features_file in dataset_split[f'{k}_test']:
            df = get_croppped_dataframe(annotated_features_file)
            # Get segments
            X_test, Y_test = get_segments(df=df)
            
            assert len(X_test) == len(df)
            
            segments['X_test'].extend(X_test)
            segments['Y_test'].extend(Y_test)
            
            # Record lengths of each batch of segments (needed for correct evaluation)
            segments['test_len'].append(len(X_test))
            
        # Convert lists to numpy arrays and reshape Y to be 3D (as needed for training)
        for key in segments.keys():
            segments[key] = np.array(segments[key])
            if key[0] == 'Y':
                segments[key] = np.expand_dims(segments[key], axis=-1)               
            print(key, segments[key].shape)
            
        # Save train/val/test segmented data for this fold
        segments['selected_features'] = selected_features
        segments['WINDOW_SIZE'] = window_size
        segments['MASK_VALUE'] = MASK_VALUE            
        np.savez(dataset_output_filename_prefix + f'{k}fold_{dataset_type}', **segments)
        
        print(f'\t FOLD {k}')
        n_examples = len(segments['X_train']) + len(segments['X_val']) + len(segments['X_test'])
        print(f"\t\t train:val:test = {len(segments['X_train'])}:{len(segments['X_val'])}:{len(segments['X_test'])} = {len(segments['X_train'])/n_examples}:{len(segments['X_val'])/n_examples}:{len(segments['X_test'])/n_examples}")
        for partition in ['train', 'val', 'test']:
            print(f'\t\t Number of {partition} examples per class: \t{np.unique(segments[f"Y_{partition}"][:, -1], return_counts=True)}')        
        print(f'\t\t Total time taken: {time.time() - start_time} s')
        print('====================================================================================================')


In [2]:
# MAIN: 45 folds (with cleaning/cutting of the beginnings)

selected_features_1 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
]
selected_features_2 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',
    
    'diff2_ pose_Tx', 
    'diff2_ pose_Ty', 
    'diff2_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
    
    'diff2_ pose_Rx', 
    'diff2_ pose_Ry', 
    'diff2_ pose_Rz',
]

for sf in [selected_features_1, selected_features_2]:
    for ws in [16, 32, 64]:
        segment_dataset(n_folds=45, selected_features=sf, window_size=ws, val_size=0.15)

Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz']
X_train (154430, 16, 6)
X_val (27224, 16, 6)
Y_train (154430, 16, 1)
Y_val (27224, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (3099, 16, 6)
Y_test (3099, 16, 1)
test_len (1,)
	 FOLD 0
		 train:val:test = 154430:27224:3099 = 0.8358727598469308:0.14735349358332475:0.01677374656974447
		 Number of train examples per class: 	(array([0., 1.]), array([134580,  19850]))
		 Number of val examples per class: 	(array([0., 1.]), array([23695,  3529]))
		 Number of test examples per class: 	(array([0., 1.]), array([3074,   25]))
		 Total time taken: 27.22671151161194 s
X_train (154988, 16, 6)
X_val (27322, 16, 6)
Y_train (154988, 16, 1)
Y_val (27322, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (2443, 16, 6)
Y_test (2443, 16, 1)
test_len (1,)
	 FOLD 1
		 train:val:test = 154988:27322:2443 = 0.8388930085032449:0.14788393151937992:0.013223059977375199
		 Number of train exa

	 FOLD 12
		 train:val:test = 155225:27364:2164 = 0.8401758022873783:0.14811126206340358:0.011712935649218145
		 Number of train examples per class: 	(array([0., 1.]), array([135834,  19391]))
		 Number of val examples per class: 	(array([0., 1.]), array([23376,  3988]))
		 Number of test examples per class: 	(array([0., 1.]), array([2139,   25]))
		 Total time taken: 344.25862979888916 s
X_train (152656, 16, 6)
X_val (26911, 16, 6)
Y_train (152656, 16, 1)
Y_val (26911, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (5186, 16, 6)
Y_test (5186, 16, 1)
test_len (1,)
	 FOLD 13
		 train:val:test = 152656:26911:5186 = 0.8262707506779322:0.14565933976714857:0.02806990955491927
		 Number of train examples per class: 	(array([0., 1.]), array([132605,  20051]))
		 Number of val examples per class: 	(array([0., 1.]), array([23558,  3353]))
		 Number of test examples per class: 	(array([0.]), array([5186]))
		 Total time taken: 372.59989738464355 s
X_train (154709, 16, 6)
X_val (27273, 16, 6)
Y_tr

X_train (154437, 16, 6)
X_val (27225, 16, 6)
Y_train (154437, 16, 1)
Y_val (27225, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (3091, 16, 6)
Y_test (3091, 16, 1)
test_len (1,)
	 FOLD 25
		 train:val:test = 154437:27225:3091 = 0.8359106482709348:0.14735890621532532:0.016730445513739967
		 Number of train examples per class: 	(array([0., 1.]), array([136027,  18410]))
		 Number of val examples per class: 	(array([0., 1.]), array([23866,  3359]))
		 Number of test examples per class: 	(array([0., 1.]), array([1456, 1635]))
		 Total time taken: 690.7060649394989 s
X_train (155030, 16, 6)
X_val (27330, 16, 6)
Y_train (155030, 16, 1)
Y_val (27330, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (2393, 16, 6)
Y_test (2393, 16, 1)
test_len (1,)
	 FOLD 26
		 train:val:test = 155030:27330:2393 = 0.8391203390472685:0.14792723257538443:0.012952428377347053
		 Number of train examples per class: 	(array([0., 1.]), array([135684,  19346]))
		 Number of val examples per class: 	(array([0., 1.]), arra

X_train (153292, 16, 6)
X_val (27023, 16, 6)
Y_train (153292, 16, 1)
Y_val (27023, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (4438, 16, 6)
Y_test (4438, 16, 1)
test_len (1,)
	 FOLD 38
		 train:val:test = 153292:27023:4438 = 0.8297131846302902:0.14626555455121162:0.024021260818498213
		 Number of train examples per class: 	(array([0., 1.]), array([133347,  19945]))
		 Number of val examples per class: 	(array([0., 1.]), array([23564,  3459]))
		 Number of test examples per class: 	(array([0.]), array([4438]))
		 Total time taken: 1038.2657878398895 s
X_train (153483, 16, 6)
X_val (27057, 16, 6)
Y_train (153483, 16, 1)
Y_val (27057, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (4213, 16, 6)
Y_test (4213, 16, 1)
test_len (1,)
	 FOLD 39
		 train:val:test = 153483:27057:4213 = 0.8307469973423977:0.14644958403923075:0.022803418618371554
		 Number of train examples per class: 	(array([0., 1.]), array([133353,  20130]))
		 Number of val examples per class: 	(array([0., 1.]), array([23783,

Y_val (26798, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (5935, 32, 6)
Y_test (5935, 32, 1)
test_len (1,)
	 FOLD 5
		 train:val:test = 152020:26798:5935 = 0.8228283167255741:0.14504771235108496:0.03212397092334089
		 Number of train examples per class: 	(array([0., 1.]), array([132522,  19498]))
		 Number of val examples per class: 	(array([0., 1.]), array([23781,  3017]))
		 Number of test examples per class: 	(array([0., 1.]), array([5046,  889]))
		 Total time taken: 162.10921430587769 s
X_train (155271, 32, 6)
X_val (27372, 32, 6)
Y_train (155271, 32, 1)
Y_val (27372, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (2110, 32, 6)
Y_test (2110, 32, 1)
test_len (1,)
	 FOLD 6
		 train:val:test = 155271:27372:2110 = 0.8404247833594042:0.14815456311940808:0.011420653521187749
		 Number of train examples per class: 	(array([0., 1.]), array([135702,  19569]))
		 Number of val examples per class: 	(array([0., 1.]), array([23537,  3835]))
		 Number of test examples per class: 	(array([0.]),

...

X_train (153382, 32, 12)
X_val (27039, 32, 12)
Y_train (153382, 32, 1)
Y_val (27039, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4332, 32, 12)
Y_test (4332, 32, 1)
test_len (1,)
	 FOLD 32
		 train:val:test = 153382:27039:4332 = 0.8302003215103408:0.14635215666322063:0.02344752182643854
		 Number of train examples per class: 	(array([0., 1.]), array([134961,  18421]))
		 Number of val examples per class: 	(array([0., 1.]), array([22381,  4658]))
		 Number of test examples per class: 	(array([0., 1.]), array([4007,  325]))
		 Total time taken: 916.2870762348175 s
X_train (153682, 32, 12)
X_val (27091, 32, 12)
Y_train (153682, 32, 1)
Y_val (27091, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3980, 32, 12)
Y_test (3980, 32, 1)
test_len (1,)
	 FOLD 33
		 train:val:test = 153682:27091:3980 = 0.8318241111105097:0.1466336135272499:0.021542275362240397
		 Number of train examples per class: 	(array([0., 1.]), array([135475,  18207]))
		 Number of val examples per class: 	(array([0., 1.]), 

	 FOLD 44
		 train:val:test = 152701:26919:5133 = 0.8265143191179575:0.14570264082315307:0.027783040058889437
		 Number of train examples per class: 	(array([0., 1.]), array([133270,  19431]))
		 Number of val examples per class: 	(array([0., 1.]), array([23188,  3731]))
		 Number of test examples per class: 	(array([0., 1.]), array([4891,  242]))
		 Total time taken: 1247.1147968769073 s
Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff2_ pose_Tx', 'diff2_ pose_Ty', 'diff2_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz', 'diff2_ pose_Rx', 'diff2_ pose_Ry', 'diff2_ pose_Rz']
X_train (154430, 64, 12)
X_val (27224, 64, 12)
Y_train (154430, 64, 1)
Y_val (27224, 64, 1)
train_len (44, 2)
val_len (44,)
X_test (3099, 64, 12)
Y_test (3099, 64, 1)
test_len (1,)
	 FOLD 0
		 train:val:test = 154430:27224:3099 = 0.8358727598469308:0.14735349358332475:0.01677374656974447
		 Number of train examples per class: 	(array([0., 1.]), array([134909,  19521]))
		 N

	 FOLD 11
		 train:val:test = 154539:27243:2971 = 0.8364627367349922:0.14745633359133545:0.016080929673672417
		 Number of train examples per class: 	(array([0., 1.]), array([134849,  19690]))
		 Number of val examples per class: 	(array([0., 1.]), array([24336,  2907]))
		 Number of test examples per class: 	(array([0., 1.]), array([2164,  807]))
		 Total time taken: 346.7652199268341 s
X_train (155225, 64, 12)
X_val (27364, 64, 12)
Y_train (155225, 64, 1)
Y_val (27364, 64, 1)
train_len (44, 2)
val_len (44,)
X_test (2164, 64, 12)
Y_test (2164, 64, 1)
test_len (1,)
	 FOLD 12
		 train:val:test = 155225:27364:2164 = 0.8401758022873783:0.14811126206340358:0.011712935649218145
		 Number of train examples per class: 	(array([0., 1.]), array([135806,  19419]))
		 Number of val examples per class: 	(array([0., 1.]), array([23404,  3960]))
		 Number of test examples per class: 	(array([0., 1.]), array([2139,   25]))
		 Total time taken: 377.5790572166443 s
X_train (152656, 64, 12)
X_val (26911

X_train (154555, 64, 12)
X_val (27246, 64, 12)
Y_train (154555, 64, 1)
Y_val (27246, 64, 1)
train_len (44, 2)
val_len (44,)
X_test (2952, 64, 12)
Y_test (2952, 64, 1)
test_len (1,)
	 FOLD 24
		 train:val:test = 154555:27246:2952 = 0.8365493388470011:0.14747257148733714:0.01597808966566172
		 Number of train examples per class: 	(array([0., 1.]), array([135606,  18949]))
		 Number of val examples per class: 	(array([0., 1.]), array([24017,  3229]))
		 Number of test examples per class: 	(array([0., 1.]), array([1726, 1226]))
		 Total time taken: 724.6283226013184 s
X_train (154437, 64, 12)
X_val (27225, 64, 12)
Y_train (154437, 64, 1)
Y_val (27225, 64, 1)
train_len (44, 2)
val_len (44,)
X_test (3091, 64, 12)
Y_test (3091, 64, 1)
test_len (1,)
	 FOLD 25
		 train:val:test = 154437:27225:3091 = 0.8359106482709348:0.14735890621532532:0.016730445513739967
		 Number of train examples per class: 	(array([0., 1.]), array([135574,  18863]))
		 Number of val examples per class: 	(array([0., 1.]),

	 FOLD 36
		 train:val:test = 153398:27041:4314 = 0.8302869236223498:0.14636298192722175:0.023350094450428408
		 Number of train examples per class: 	(array([0., 1.]), array([133776,  19622]))
		 Number of val examples per class: 	(array([0., 1.]), array([23259,  3782]))
		 Number of test examples per class: 	(array([0.]), array([4314]))
		 Total time taken: 1076.5707087516785 s
X_train (152706, 64, 12)
X_val (26919, 64, 12)
Y_train (152706, 64, 1)
Y_val (26919, 64, 1)
train_len (44, 2)
val_len (44,)
X_test (5128, 64, 12)
Y_test (5128, 64, 1)
test_len (1,)
	 FOLD 37
		 train:val:test = 152706:26919:5128 = 0.8265413822779603:0.14570264082315307:0.027755976898886623
		 Number of train examples per class: 	(array([0., 1.]), array([133753,  18953]))
		 Number of val examples per class: 	(array([0., 1.]), array([24375,  2544]))
		 Number of test examples per class: 	(array([0., 1.]), array([3221, 1907]))
		 Total time taken: 1106.6962921619415 s
X_train (153292, 64, 12)
X_val (27023, 64, 12

In [2]:
# MAIN: 45 folds (without cleaning/cutting of the beginnings)

selected_features_1 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
]
selected_features_2 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',
    
    'diff2_ pose_Tx', 
    'diff2_ pose_Ty', 
    'diff2_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
    
    'diff2_ pose_Rx', 
    'diff2_ pose_Ry', 
    'diff2_ pose_Rz',
]

for sf in [selected_features_1, selected_features_2]:
    for ws in [8, 16, 32, 64]:
        segment_dataset(n_folds=45, selected_features=sf, window_size=ws, val_size=0.15)

Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz']
X_train (159911, 8, 6)
X_val (28192, 8, 6)
Y_train (159911, 8, 1)
Y_val (28192, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (3278, 8, 6)
Y_test (3278, 8, 1)
test_len (1,)
	 FOLD 0
		 train:val:test = 159911:28192:3278 = 0.8355636139428679:0.14730824899023415:0.017128137066897968
		 Number of train examples per class: 	(array([0., 1.]), array([139074,  20837]))
		 Number of val examples per class: 	(array([0., 1.]), array([25650,  2542]))
		 Number of test examples per class: 	(array([0., 1.]), array([3253,   25]))
		 Total time taken: 34.30186891555786 s
X_train (160597, 8, 6)
X_val (28313, 8, 6)
Y_train (160597, 8, 1)
Y_val (28313, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (2471, 8, 6)
Y_test (2471, 8, 1)
test_len (1,)
	 FOLD 1
		 train:val:test = 160597:28313:2471 = 0.8391480868006751:0.14794049566048875:0.012911417538836144
		 Number of train examples per c

	 FOLD 12
		 train:val:test = 160858:28359:2164 = 0.8405118585439516:0.14818085389876737:0.011307287557281026
		 Number of train examples per class: 	(array([0., 1.]), array([141063,  19795]))
		 Number of val examples per class: 	(array([0., 1.]), array([24775,  3584]))
		 Number of test examples per class: 	(array([0., 1.]), array([2139,   25]))
		 Total time taken: 440.4616689682007 s
X_train (158289, 8, 6)
X_val (27906, 8, 6)
Y_train (158289, 8, 1)
Y_val (27906, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (5186, 8, 6)
Y_test (5186, 8, 1)
test_len (1,)
	 FOLD 13
		 train:val:test = 158289:27906:5186 = 0.8270883734540001:0.1458138477696323:0.02709777877636756
		 Number of train examples per class: 	(array([0., 1.]), array([138365,  19924]))
		 Number of val examples per class: 	(array([0., 1.]), array([24426,  3480]))
		 Number of test examples per class: 	(array([0.]), array([5186]))
		 Total time taken: 473.6910152435303 s
X_train (160075, 8, 6)
X_val (28221, 8, 6)
Y_train (160075

X_train (159990, 8, 6)
X_val (28206, 8, 6)
Y_train (159990, 8, 1)
Y_val (28206, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (3185, 8, 6)
Y_test (3185, 8, 1)
test_len (1,)
	 FOLD 25
		 train:val:test = 159990:28206:3185 = 0.8359764030912159:0.14738140149753634:0.016642195411247722
		 Number of train examples per class: 	(array([0., 1.]), array([140991,  18999]))
		 Number of val examples per class: 	(array([0., 1.]), array([25436,  2770]))
		 Number of test examples per class: 	(array([0., 1.]), array([1550, 1635]))
		 Total time taken: 886.4225792884827 s
X_train (160641, 8, 6)
X_val (28320, 8, 6)
Y_train (160641, 8, 1)
Y_val (28320, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (2420, 8, 6)
Y_test (2420, 8, 1)
test_len (1,)
	 FOLD 26
		 train:val:test = 160641:28320:2420 = 0.8393779946807677:0.14797707191413986:0.01264493340509246
		 Number of train examples per class: 	(array([0., 1.]), array([141593,  19048]))
		 Number of val examples per class: 	(array([0., 1.]), array([24368,  39

X_train (158585, 8, 6)
X_val (27958, 8, 6)
Y_train (158585, 8, 1)
Y_val (27958, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (4838, 8, 6)
Y_test (4838, 8, 1)
test_len (1,)
	 FOLD 38
		 train:val:test = 158585:27958:4838 = 0.8286350264655321:0.146085557082469:0.025279416451998892
		 Number of train examples per class: 	(array([0., 1.]), array([138367,  20218]))
		 Number of val examples per class: 	(array([0., 1.]), array([24772,  3186]))
		 Number of test examples per class: 	(array([0.]), array([4838]))
		 Total time taken: 1355.567091703415 s
X_train (159116, 8, 6)
X_val (28052, 8, 6)
Y_train (159116, 8, 1)
Y_val (28052, 8, 1)
train_len (44, 2)
val_len (44,)
X_test (4213, 8, 6)
Y_test (4213, 8, 1)
test_len (1,)
	 FOLD 39
		 train:val:test = 159116:28052:4213 = 0.8314095965639222:0.14657672391721227:0.022013679518865507
		 Number of train examples per class: 	(array([0., 1.]), array([139117,  19999]))
		 Number of val examples per class: 	(array([0., 1.]), array([24647,  3405]))
		 Nu

	 FOLD 5
		 train:val:test = 157649:27793:5939 = 0.8237442588344716:0.1452234025321218:0.03103233863340666
		 Number of train examples per class: 	(array([0., 1.]), array([138577,  19072]))
		 Number of val examples per class: 	(array([0., 1.]), array([24350,  3443]))
		 Number of test examples per class: 	(array([0., 1.]), array([5050,  889]))
		 Total time taken: 215.03338265419006 s
X_train (160904, 16, 6)
X_val (28367, 16, 6)
Y_train (160904, 16, 1)
Y_val (28367, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (2110, 16, 6)
Y_test (2110, 16, 1)
test_len (1,)
	 FOLD 6
		 train:val:test = 160904:28367:2110 = 0.8407522167822302:0.1482226553315115:0.011025127886258302
		 Number of train examples per class: 	(array([0., 1.]), array([140683,  20221]))
		 Number of val examples per class: 	(array([0., 1.]), array([25184,  3183]))
		 Number of test examples per class: 	(array([0.]), array([2110]))
		 Total time taken: 250.1830072402954 s
X_train (157915, 16, 6)
X_val (27840, 16, 6)
Y_train (

...

	 FOLD 30
		 train:val:test = 160594:28312:2475 = 0.8391324112633961:0.14793527048139574:0.012932318255208197
		 Number of train examples per class: 	(array([0., 1.]), array([140210,  20384]))
		 Number of val examples per class: 	(array([0., 1.]), array([25384,  2928]))
		 Number of test examples per class: 	(array([0., 1.]), array([2383,   92]))
		 Total time taken: 1170.7942469120026 s
X_train (159256, 16, 6)
X_val (28076, 16, 6)
Y_train (159256, 16, 1)
Y_val (28076, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (4049, 16, 6)
Y_test (4049, 16, 1)
test_len (1,)
	 FOLD 31
		 train:val:test = 159256:28076:4049 = 0.8321411216369441:0.1467021282154446:0.02115675014761131
		 Number of train examples per class: 	(array([0., 1.]), array([139616,  19640]))
		 Number of val examples per class: 	(array([0., 1.]), array([24439,  3637]))
		 Number of test examples per class: 	(array([0., 1.]), array([3922,  127]))
		 Total time taken: 1206.070497751236 s
X_train (158861, 16, 6)
X_val (28006, 16,

X_train (159939, 16, 6)
X_val (28197, 16, 6)
Y_train (159939, 16, 1)
Y_val (28197, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (3245, 16, 6)
Y_test (3245, 16, 1)
test_len (1,)
	 FOLD 43
		 train:val:test = 159939:28197:3245 = 0.8357099189574723:0.1473343748856992:0.016955706156828527
		 Number of train examples per class: 	(array([0., 1.]), array([140599,  19340]))
		 Number of val examples per class: 	(array([0., 1.]), array([24578,  3619]))
		 Number of test examples per class: 	(array([0., 1.]), array([2800,  445]))
		 Total time taken: 1636.248127937317 s
X_train (157994, 16, 6)
X_val (27853, 16, 6)
Y_train (157994, 16, 1)
Y_val (27853, 16, 1)
train_len (44, 2)
val_len (44,)
X_test (5534, 16, 6)
Y_test (5534, 16, 1)
test_len (1,)
	 FOLD 44
		 train:val:test = 157994:27853:5534 = 0.8255469456215612:0.1455369132777026:0.028916141100736228
		 Number of train examples per class: 	(array([0., 1.]), array([137545,  20449]))
		 Number of val examples per class: 	(array([0., 1.]), array(

X_test (3983, 32, 6)
Y_test (3983, 32, 1)
test_len (1,)
	 FOLD 10
		 train:val:test = 159312:28086:3983 = 0.8324337316661529:0.14675438000637472:0.020811888327472425
		 Number of train examples per class: 	(array([0., 1.]), array([141319,  17993]))
		 Number of val examples per class: 	(array([0., 1.]), array([23911,  4175]))
		 Number of test examples per class: 	(array([0., 1.]), array([2747, 1236]))
		 Total time taken: 396.5307755470276 s
X_train (159684, 32, 6)
X_val (28152, 32, 6)
Y_train (159684, 32, 1)
Y_val (28152, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3545, 32, 6)
Y_test (3545, 32, 1)
test_len (1,)
	 FOLD 11
		 train:val:test = 159684:28152:3545 = 0.8343774982887538:0.1470992418265136:0.018523259884732548
		 Number of train examples per class: 	(array([0., 1.]), array([139389,  20295]))
		 Number of val examples per class: 	(array([0., 1.]), array([25850,  2302]))
		 Number of test examples per class: 	(array([0., 1.]), array([2738,  807]))
		 Total time taken: 431.9

X_train (160924, 32, 6)
X_val (28370, 32, 6)
Y_train (160924, 32, 1)
Y_val (28370, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (2087, 32, 6)
Y_test (2087, 32, 1)
test_len (1,)
	 FOLD 23
		 train:val:test = 160924:28370:2087 = 0.8408567203640904:0.14823833086879054:0.010904948767118993
		 Number of train examples per class: 	(array([0., 1.]), array([142115,  18809]))
		 Number of val examples per class: 	(array([0., 1.]), array([25006,  3364]))
		 Number of test examples per class: 	(array([0., 1.]), array([ 856, 1231]))
		 Total time taken: 859.5560538768768 s
X_train (160113, 32, 6)
X_val (28227, 32, 6)
Y_train (160113, 32, 1)
Y_val (28227, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3041, 32, 6)
Y_test (3041, 32, 1)
test_len (1,)
	 FOLD 24
		 train:val:test = 160113:28227:3041 = 0.8366191001196566:0.14749113025848962:0.01588976962185379
		 Number of train examples per class: 	(array([0., 1.]), array([141720,  18393]))
		 Number of val examples per class: 	(array([0., 1.]), array

OSError: [Errno 28] No space left on device

In [2]:
# Generate dataset without translations (dataset_type: R_...)

selected_features_1 = [
    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz'
]

selected_features_2 = [
    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',

    'diff2_ pose_Rx', 
    'diff2_ pose_Ry', 
    'diff2_ pose_Rz'
]

for sf in [selected_features_1, selected_features_2]:
    for ws in [32]:
        segment_dataset(n_folds=45, selected_features=sf, window_size=ws, val_size=0.15)

Selected features: 
	['diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz']
X_train (159911, 32, 3)
X_val (28192, 32, 3)
Y_train (159911, 32, 1)
Y_val (28192, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3278, 32, 3)
Y_test (3278, 32, 1)
test_len (1,)
	 FOLD 0
		 train:val:test = 159911:28192:3278 = 0.8355636139428679:0.14730824899023415:0.017128137066897968
		 Number of train examples per class: 	(array([0., 1.]), array([139074,  20837]))
		 Number of val examples per class: 	(array([0., 1.]), array([25650,  2542]))
		 Number of test examples per class: 	(array([0., 1.]), array([3253,   25]))
		 Total time taken: 20.748727083206177 s
X_train (160597, 32, 3)
X_val (28313, 32, 3)
Y_train (160597, 32, 1)
Y_val (28313, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (2471, 32, 3)
Y_test (2471, 32, 1)
test_len (1,)
	 FOLD 1
		 train:val:test = 160597:28313:2471 = 0.8391480868006751:0.14794049566048875:0.012911417538836144
		 Number of train examples per class: 	(array([0., 1.]), array([140748

	 FOLD 12
		 train:val:test = 160858:28359:2164 = 0.8405118585439516:0.14818085389876737:0.011307287557281026
		 Number of train examples per class: 	(array([0., 1.]), array([141063,  19795]))
		 Number of val examples per class: 	(array([0., 1.]), array([24775,  3584]))
		 Number of test examples per class: 	(array([0., 1.]), array([2139,   25]))
		 Total time taken: 280.97835516929626 s
X_train (158289, 32, 3)
X_val (27906, 32, 3)
Y_train (158289, 32, 1)
Y_val (27906, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (5186, 32, 3)
Y_test (5186, 32, 1)
test_len (1,)
	 FOLD 13
		 train:val:test = 158289:27906:5186 = 0.8270883734540001:0.1458138477696323:0.02709777877636756
		 Number of train examples per class: 	(array([0., 1.]), array([138365,  19924]))
		 Number of val examples per class: 	(array([0., 1.]), array([24426,  3480]))
		 Number of test examples per class: 	(array([0.]), array([5186]))
		 Total time taken: 303.45824122428894 s
X_train (160075, 32, 3)
X_val (28221, 32, 3)
Y_tra

X_train (159990, 32, 3)
X_val (28206, 32, 3)
Y_train (159990, 32, 1)
Y_val (28206, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3185, 32, 3)
Y_test (3185, 32, 1)
test_len (1,)
	 FOLD 25
		 train:val:test = 159990:28206:3185 = 0.8359764030912159:0.14738140149753634:0.016642195411247722
		 Number of train examples per class: 	(array([0., 1.]), array([140991,  18999]))
		 Number of val examples per class: 	(array([0., 1.]), array([25436,  2770]))
		 Number of test examples per class: 	(array([0., 1.]), array([1550, 1635]))
		 Total time taken: 564.3162086009979 s
X_train (160641, 32, 3)
X_val (28320, 32, 3)
Y_train (160641, 32, 1)
Y_val (28320, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (2420, 32, 3)
Y_test (2420, 32, 1)
test_len (1,)
	 FOLD 26
		 train:val:test = 160641:28320:2420 = 0.8393779946807677:0.14797707191413986:0.01264493340509246
		 Number of train examples per class: 	(array([0., 1.]), array([141593,  19048]))
		 Number of val examples per class: 	(array([0., 1.]), array

X_train (158585, 32, 3)
X_val (27958, 32, 3)
Y_train (158585, 32, 1)
Y_val (27958, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4838, 32, 3)
Y_test (4838, 32, 1)
test_len (1,)
	 FOLD 38
		 train:val:test = 158585:27958:4838 = 0.8286350264655321:0.146085557082469:0.025279416451998892
		 Number of train examples per class: 	(array([0., 1.]), array([138367,  20218]))
		 Number of val examples per class: 	(array([0., 1.]), array([24772,  3186]))
		 Number of test examples per class: 	(array([0.]), array([4838]))
		 Total time taken: 843.651974439621 s
X_train (159116, 32, 3)
X_val (28052, 32, 3)
Y_train (159116, 32, 1)
Y_val (28052, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4213, 32, 3)
Y_test (4213, 32, 1)
test_len (1,)
	 FOLD 39
		 train:val:test = 159116:28052:4213 = 0.8314095965639222:0.14657672391721227:0.022013679518865507
		 Number of train examples per class: 	(array([0., 1.]), array([139117,  19999]))
		 Number of val examples per class: 	(array([0., 1.]), array([24647,  34

	 FOLD 5
		 train:val:test = 157649:27793:5939 = 0.8237442588344716:0.1452234025321218:0.03103233863340666
		 Number of train examples per class: 	(array([0., 1.]), array([138577,  19072]))
		 Number of val examples per class: 	(array([0., 1.]), array([24350,  3443]))
		 Number of test examples per class: 	(array([0., 1.]), array([5050,  889]))
		 Total time taken: 136.36892914772034 s
X_train (160904, 32, 6)
X_val (28367, 32, 6)
Y_train (160904, 32, 1)
Y_val (28367, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (2110, 32, 6)
Y_test (2110, 32, 1)
test_len (1,)
	 FOLD 6
		 train:val:test = 160904:28367:2110 = 0.8407522167822302:0.1482226553315115:0.011025127886258302
		 Number of train examples per class: 	(array([0., 1.]), array([140683,  20221]))
		 Number of val examples per class: 	(array([0., 1.]), array([25184,  3183]))
		 Number of test examples per class: 	(array([0.]), array([2110]))
		 Total time taken: 160.11165595054626 s
X_train (157915, 32, 6)
X_val (27840, 32, 6)
Y_train 

X_train (158884, 32, 6)
X_val (28010, 32, 6)
Y_train (158884, 32, 1)
Y_val (28010, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4487, 32, 6)
Y_test (4487, 32, 1)
test_len (1,)
	 FOLD 18
		 train:val:test = 158884:28010:4487 = 0.8301973550143431:0.1463572663953057:0.023445378590351184
		 Number of train examples per class: 	(array([0., 1.]), array([139413,  19471]))
		 Number of val examples per class: 	(array([0., 1.]), array([24077,  3933]))
		 Number of test examples per class: 	(array([0.]), array([4487]))
		 Total time taken: 433.427127122879 s
X_train (159258, 32, 6)
X_val (28076, 32, 6)
Y_train (159258, 32, 1)
Y_val (28076, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4047, 32, 6)
Y_test (4047, 32, 1)
test_len (1,)
	 FOLD 19
		 train:val:test = 159258:28076:4047 = 0.8321515719951301:0.1467021282154446:0.021146299789425282
		 Number of train examples per class: 	(array([0., 1.]), array([140215,  19043]))
		 Number of val examples per class: 	(array([0., 1.]), array([24122,  39

	 FOLD 30
		 train:val:test = 160594:28312:2475 = 0.8391324112633961:0.14793527048139574:0.012932318255208197
		 Number of train examples per class: 	(array([0., 1.]), array([140210,  20384]))
		 Number of val examples per class: 	(array([0., 1.]), array([25384,  2928]))
		 Number of test examples per class: 	(array([0., 1.]), array([2383,   92]))
		 Total time taken: 698.8032219409943 s
X_train (159256, 32, 6)
X_val (28076, 32, 6)
Y_train (159256, 32, 1)
Y_val (28076, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (4049, 32, 6)
Y_test (4049, 32, 1)
test_len (1,)
	 FOLD 31
		 train:val:test = 159256:28076:4049 = 0.8321411216369441:0.1467021282154446:0.02115675014761131
		 Number of train examples per class: 	(array([0., 1.]), array([139616,  19640]))
		 Number of val examples per class: 	(array([0., 1.]), array([24439,  3637]))
		 Number of test examples per class: 	(array([0., 1.]), array([3922,  127]))
		 Total time taken: 723.4325649738312 s
X_train (158861, 32, 6)
X_val (28006, 32, 

X_train (159939, 32, 6)
X_val (28197, 32, 6)
Y_train (159939, 32, 1)
Y_val (28197, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (3245, 32, 6)
Y_test (3245, 32, 1)
test_len (1,)
	 FOLD 43
		 train:val:test = 159939:28197:3245 = 0.8357099189574723:0.1473343748856992:0.016955706156828527
		 Number of train examples per class: 	(array([0., 1.]), array([140599,  19340]))
		 Number of val examples per class: 	(array([0., 1.]), array([24578,  3619]))
		 Number of test examples per class: 	(array([0., 1.]), array([2800,  445]))
		 Total time taken: 1011.3109033107758 s
X_train (157994, 32, 6)
X_val (27853, 32, 6)
Y_train (157994, 32, 1)
Y_val (27853, 32, 1)
train_len (44, 2)
val_len (44,)
X_test (5534, 32, 6)
Y_test (5534, 32, 1)
test_len (1,)
	 FOLD 44
		 train:val:test = 157994:27853:5534 = 0.8255469456215612:0.1455369132777026:0.028916141100736228
		 Number of train examples per class: 	(array([0., 1.]), array([137545,  20449]))
		 Number of val examples per class: 	(array([0., 1.]), array

In [2]:
# 9 folds

selected_features_1 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
]
selected_features_2 = [
    'diff_ pose_Tx', 
    'diff_ pose_Ty', 
    'diff_ pose_Tz',
    
    'diff2_ pose_Tx', 
    'diff2_ pose_Ty', 
    'diff2_ pose_Tz',

    'diff_ pose_Rx', 
    'diff_ pose_Ry', 
    'diff_ pose_Rz',
    
    'diff2_ pose_Rx', 
    'diff2_ pose_Ry', 
    'diff2_ pose_Rz',
]

for sf in [selected_features_1, selected_features_2]:
    for ws in [8, 16, 32, 64]:
        segment_dataset(n_folds=9, selected_features=sf, window_size=ws, val_size=0.15)
        

Selected features: 
	['diff_ pose_Tx', 'diff_ pose_Ty', 'diff_ pose_Tz', 'diff_ pose_Rx', 'diff_ pose_Ry', 'diff_ pose_Rz']
X_train (145797, 8, 6)
X_val (25704, 8, 6)
Y_train (145797, 8, 1)
Y_val (25704, 8, 1)
train_len (40, 2)
val_len (40,)
X_test (19880, 8, 6)
Y_test (19880, 8, 1)
test_len (5,)
	 FOLD 0
		 train:val:test = 145797:25704:19880 = 0.7618154362240765:0.13430800340681676:0.10387656036910665
		 Number of train examples per class: 	(array([0., 1.]), array([127814,  17983]))
		 Number of val examples per class: 	(array([0., 1.]), array([23181,  2523]))
		 Number of test examples per class: 	(array([0., 1.]), array([16982,  2898]))
		 Total time taken: 35.215213775634766 s
X_train (141170, 8, 6)
X_val (24888, 8, 6)
Y_train (141170, 8, 1)
Y_val (24888, 8, 1)
train_len (40, 2)
val_len (40,)
X_test (25323, 8, 6)
Y_test (25323, 8, 1)
test_len (5,)
	 FOLD 1
		 train:val:test = 141170:24888:25323 = 0.7376385325607036:0.13004425726691782:0.13231721017237866
		 Number of train example

X_train (141802, 16, 6)
X_val (24998, 16, 6)
Y_train (141802, 16, 1)
Y_val (24998, 16, 1)
train_len (40, 2)
val_len (40,)
X_test (24581, 16, 6)
Y_test (24581, 16, 1)
test_len (5,)
	 FOLD 3
		 train:val:test = 141802:24998:24581 = 0.740940845747488:0.1306190269671493:0.1284401272853627
		 Number of train examples per class: 	(array([0., 1.]), array([122928,  18874]))
		 Number of val examples per class: 	(array([0., 1.]), array([22070,  2928]))
		 Number of test examples per class: 	(array([0., 1.]), array([22979,  1602]))
		 Total time taken: 137.21652007102966 s
X_train (145361, 16, 6)
X_val (25625, 16, 6)
Y_train (145361, 16, 1)
Y_val (25625, 16, 1)
train_len (40, 2)
val_len (40,)
X_test (20395, 16, 6)
Y_test (20395, 16, 1)
test_len (5,)
	 FOLD 4
		 train:val:test = 145361:25625:20395 = 0.7595372581395228:0.1338952142584687:0.10656752760200856
		 Number of train examples per class: 	(array([0., 1.]), array([129348,  16013]))
		 Number of val examples per class: 	(array([0., 1.]), arr

X_test (20599, 32, 6)
Y_test (20599, 32, 1)
test_len (5,)
	 FOLD 6
		 train:val:test = 145187:25595:20599 = 0.7586280769773384:0.1337384588856783:0.1076334641369833
		 Number of train examples per class: 	(array([0., 1.]), array([126726,  18461]))
		 Number of val examples per class: 	(array([0., 1.]), array([22495,  3100]))
		 Number of test examples per class: 	(array([0., 1.]), array([18756,  1843]))
		 Total time taken: 245.55152559280396 s
X_train (144115, 32, 6)
X_val (25408, 32, 6)
Y_train (144115, 32, 1)
Y_val (25408, 32, 1)
train_len (40, 2)
val_len (40,)
X_test (21858, 32, 6)
Y_test (21858, 32, 1)
test_len (5,)
	 FOLD 7
		 train:val:test = 144115:25408:21858 = 0.753026684989628:0.1327613503952848:0.11421196461508719
		 Number of train examples per class: 	(array([0., 1.]), array([126248,  17867]))
		 Number of val examples per class: 	(array([0., 1.]), array([21778,  3630]))
		 Number of test examples per class: 	(array([0., 1.]), array([19951,  1907]))
		 Total time taken: 2

X_train (145797, 8, 12)
X_val (25704, 8, 12)
Y_train (145797, 8, 1)
Y_val (25704, 8, 1)
train_len (40, 2)
val_len (40,)
X_test (19880, 8, 12)
Y_test (19880, 8, 1)
test_len (5,)
	 FOLD 0
		 train:val:test = 145797:25704:19880 = 0.7618154362240765:0.13430800340681676:0.10387656036910665
		 Number of train examples per class: 	(array([0., 1.]), array([128815,  16982]))
		 Number of val examples per class: 	(array([0., 1.]), array([22180,  3524]))
		 Number of test examples per class: 	(array([0., 1.]), array([16982,  2898]))
		 Total time taken: 34.667014598846436 s
X_train (141170, 8, 12)
X_val (24888, 8, 12)
Y_train (141170, 8, 1)
Y_val (24888, 8, 1)
train_len (40, 2)
val_len (40,)
X_test (25323, 8, 12)
Y_test (25323, 8, 1)
test_len (5,)
	 FOLD 1
		 train:val:test = 141170:24888:25323 = 0.7376385325607036:0.13004425726691782:0.13231721017237866
		 Number of train examples per class: 	(array([0., 1.]), array([122913,  18257]))
		 Number of val examples per class: 	(array([0., 1.]), array

X_train (141802, 16, 12)
X_val (24998, 16, 12)
Y_train (141802, 16, 1)
Y_val (24998, 16, 1)
train_len (40, 2)
val_len (40,)
X_test (24581, 16, 12)
Y_test (24581, 16, 1)
test_len (5,)
	 FOLD 3
		 train:val:test = 141802:24998:24581 = 0.740940845747488:0.1306190269671493:0.1284401272853627
		 Number of train examples per class: 	(array([0., 1.]), array([123524,  18278]))
		 Number of val examples per class: 	(array([0., 1.]), array([21474,  3524]))
		 Number of test examples per class: 	(array([0., 1.]), array([22979,  1602]))
		 Total time taken: 139.26419377326965 s
X_train (145361, 16, 12)
X_val (25625, 16, 12)
Y_train (145361, 16, 1)
Y_val (25625, 16, 1)
train_len (40, 2)
val_len (40,)
X_test (20395, 16, 12)
Y_test (20395, 16, 1)
test_len (5,)
	 FOLD 4
		 train:val:test = 145361:25625:20395 = 0.7595372581395228:0.1338952142584687:0.10656752760200856
		 Number of train examples per class: 	(array([0., 1.]), array([129633,  15728]))
		 Number of val examples per class: 	(array([0., 1.]

X_train (145187, 32, 12)
X_val (25595, 32, 12)
Y_train (145187, 32, 1)
Y_val (25595, 32, 1)
train_len (40, 2)
val_len (40,)
X_test (20599, 32, 12)
Y_test (20599, 32, 1)
test_len (5,)
	 FOLD 6
		 train:val:test = 145187:25595:20599 = 0.7586280769773384:0.1337384588856783:0.1076334641369833
		 Number of train examples per class: 	(array([0., 1.]), array([126325,  18862]))
		 Number of val examples per class: 	(array([0., 1.]), array([22896,  2699]))
		 Number of test examples per class: 	(array([0., 1.]), array([18756,  1843]))
		 Total time taken: 249.606187582016 s
X_train (144115, 32, 12)
X_val (25408, 32, 12)
Y_train (144115, 32, 1)
Y_val (25408, 32, 1)
train_len (40, 2)
val_len (40,)
X_test (21858, 32, 12)
Y_test (21858, 32, 1)
test_len (5,)
	 FOLD 7
		 train:val:test = 144115:25408:21858 = 0.753026684989628:0.1327613503952848:0.11421196461508719
		 Number of train examples per class: 	(array([0., 1.]), array([125830,  18285]))
		 Number of val examples per class: 	(array([0., 1.]),